# Fermi-Hubbard Model I

In [1]:
using Pkg; Pkg.activate()

using KadanoffBaym

using FFTW, Interpolations
function wigner_transform_itp(x::AbstractMatrix, ts::Vector; fourier = true, ts_lin = range(first(ts), last(ts); length = length(ts)))
    itp = interpolate((ts, ts), x, Gridded(Linear()))
    return wigner_transform([itp(t1, t2) for t1 in ts_lin, t2 in ts_lin]; ts = ts_lin, fourier = fourier)
end

using LinearAlgebra, BlockArrays

using JLD

using PyPlot
# PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

  Activating project at `~/.julia/environments/v1.10`
Precompiling packages...
   1783.3 ms  ✓ SpecialMatrices
   1612.7 ms  ✓ KadanoffBaym
  2 dependencies successfully precompiled in 6 seconds. 29 already precompiled.
Precompiling packages...
    632.5 ms  ✓ Adapt → AdaptStaticArraysExt
    633.2 ms  ✓ StaticArrays → StaticArraysChainRulesCoreExt
   2330.8 ms  ✓ Interpolations
  3 dependencies successfully precompiled in 3 seconds. 19 already precompiled.
Precompiling packages...
    541.9 ms  ✓ Hwloc_jll
    547.4 ms  ✓ MPIPreferences
    545.9 ms  ✓ OpenSSL_jll
    543.4 ms  ✓ Lz4_jll
    539.4 ms  ✓ Zstd_jll
    573.4 ms  ✓ libaec_jll
    514.5 ms  ✓ OpenMPI_jll
    515.6 ms  ✓ MPItrampoline_jll
    522.5 ms  ✓ Blosc_jll
    619.4 ms  ✓ MPICH_jll
    465.5 ms  ✓ Blosc
   1004.4 ms  ✓ HDF5_jll
   6336.7 ms  ✓ HDF5
    857.2 ms  ✓ H5Zblosc
   1746.9 ms  ✓ JLD
  15 dependencies successfully precompiled in 11 seconds. 10 already precompiled.


## Model

### Hamiltonian

$$
\begin{align}\begin{split}
    \hat{H} &= - J \sum_{\langle{i,\,j}\rangle}\sum_\sigma \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{i+1,\sigma} + U\sum_{i=1}^L  \hat{c}^{\dagger}_{i,\uparrow} \hat{c}^{\phantom{\dagger}}_{i,\uparrow}   \hat{c}^{\dagger}_{i,\downarrow} \hat{c}^{\phantom{\dagger}}_{i,\downarrow}, 
\end{split}\end{align}
$$

### Green functions

$$
    G^>_{\uparrow,ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\uparrow}(t) \hat{c}^{{\dagger}}_{i,\uparrow}(t') \right\rangle\\
    G^>_{\downarrow,ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\downarrow}(t) \hat{c}^{{\dagger}}_{i,\downarrow}(t') \right\rangle\\
$$

### Self-energies

Hartree-Fock:
$$
    \Sigma^{\mathrm{HF}}_{\uparrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{\downarrow,ii}(t, t)\\
    \Sigma^{\mathrm{HF}}_{\downarrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{\uparrow,ii}(t, t)
$$


Second-order Born approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = U^2 G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = U^2 G_{ij, \downarrow}(t, t') G_{ij, \uparrow}(t, t') G_{ji, \uparrow}(t', t)
$$

## Solving

In [2]:
function integrate1(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t1)
    retval = zero(A[t1,t1])

    @inbounds for k in 1:tmax
        @views LinearAlgebra.mul!(retval, A[t1, k] - B[t1, k], C[k, t2], hs[k], 1.0)
    end
    return retval
end

function integrate2(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t2)
    retval = zero(A[t1,t1])

    @inbounds for k in 1:tmax
        @views LinearAlgebra.mul!(retval, A[t1, k], B[k, t2] - C[k, t2], hs[k], 1.0)
    end
    return retval
end

integrate2 (generic function with 1 method)

In [3]:
# Lattice size
L = 8

# Allocate the initial Green functions (time arguments at the end)
GL_u = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG_u = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GL_d = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG_d = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)

# Initial conditions
N_u = zeros(L)
N_d = zeros(L)

# From the paper
N_u[1:4] = [0.7, 0.0, 0.7, 0.0]
N_d[1:4] = [0.0, 0.25, 0.0, 0.25]

N_u[5:8] = [0.0, 0.4, 0.0, 0.4]
N_d[5:8] = [0.65, 0.0, 0.65, 0.0]

# From the docs
# N_u[1:4] = 0.1 .* [1, 1, 1, 1]
# N_d[1:4] = 0.1 .* [1, 1, 1, 1]

# N_u[5:8] = 0.0 .* [1, 1, 1, 1]
# N_d[5:8] = 0.0 .* [1, 1, 1, 1]

GL_u[1, 1] = 1.0im * diagm(N_u)
GG_u[1, 1] = -1.0im * (I - diagm(N_u))
GL_d[1, 1] = 1.0im * diagm(N_d)
GG_d[1, 1] = -1.0im * (I - diagm(N_d));

In [4]:
Base.@kwdef struct FermiHubbardData2B{T}
    GL_u::T
    GG_u::T
    GL_d::T
    GG_d::T

    ΣL_u::T = zero(GL_u)
    ΣG_u::T = zero(GG_u)
    ΣL_d::T = zero(GL_d)
    ΣG_d::T = zero(GG_d)
end

data = FermiHubbardData2B(GL_u=GL_u, GG_u=GG_u, GL_d=GL_d, GG_d=GG_d);

In [5]:
Base.@kwdef struct FermiHubbardModel{T}
    # interaction strength
    U::T

    # 8-site 3D cubic lattice
    h = begin
        h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
        diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
        setblock!(h, diag_block, 1, 1)
        setblock!(h, diag_block, 2, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

        h |> Array
    end

    H_u = h
    H_d = h
end

# Relatively small interaction parameter
const U₀ = 0.25
model = FermiHubbardModel(U = t -> U₀);

In [ ]:
# Right-hand side for the "vertical" evolution
function fv!(model, data, out, times, h1, h2, t, t′)
    # Unpack data and model
    (; GL_u, GG_u, GL_d, GG_d, ΣL_u, ΣG_u, ΣL_d, ΣG_d) = data
    (; H_u, H_d, U) = model

    # Real-time collision integrals
    ∫dt1(A, B, C) = integrate1(h1, t, t′, A, B, C)
    ∫dt2(A, B, C) = integrate2(h2, t, t′, A, B, C)
    
    # The interaction varies as a function of the forward time (t+t')/2
    U_t = U((times[t] + times[t′])/2)
    
    # Hartree-Fock self-energies
    ΣHF_u(t, t′) = im * U_t * Diagonal(GL_d[t, t])
    ΣHF_d(t, t′) = im * U_t * Diagonal(GL_u[t, t])
    
    # Equations of motion
    out[1] = -1.0im * ((H_u + ΣHF_u(t, t′)) * GL_u[t, t′] + 
            ∫dt1(ΣG_u, ΣL_u, GL_u) + ∫dt2(ΣL_u, GL_u, GG_u)
        )

    out[2] = -1.0im * ((H_u + ΣHF_u(t, t′)) * GG_u[t, t′] + 
            ∫dt1(ΣG_u, ΣL_u, GG_u) + ∫dt2(ΣG_u, GL_u, GG_u)
        )

    out[3] = -1.0im * ((H_d + ΣHF_d(t, t′)) * GL_d[t, t′] + 
            ∫dt1(ΣG_d, ΣL_d, GL_d) + ∫dt2(ΣL_d, GL_d, GG_d)
        )

    out[4] = -1.0im * ((H_d + ΣHF_d(t, t′)) * GG_d[t, t′] +
            ∫dt1(ΣG_d, ΣL_d, GG_d) + ∫dt2(ΣG_d, GL_d, GG_d)
        )  
    
    return out
end

# Right-hand side for the "diagonal" evolution
function fd!(model, data, out, times, h1, h2, t, t′)
    fv!(model, data, out, times, h1, h2, t, t)
    out .-= adjoint.(out)
end

fd! (generic function with 1 method)

In [11]:
# Callback function for the self-energies
function second_Born!(model, data, times, _, _, t, t′)
    # Unpack data and model
    (; GL_u, GG_u, GL_d, GG_d, ΣL_u, ΣG_u, ΣL_d, ΣG_d) = data
    (; U) = model
        
    # Resize self-energies when Green functions are resized    
    if (n = size(GL_u, 3)) > size(ΣL_u, 3)
        resize!(ΣL_u, n)
        resize!(ΣG_u, n)
        resize!(ΣL_d, n)
        resize!(ΣG_d, n)        
    end
    
    # The interaction varies as a function of the forward time (t+t')/2
    U_t = U((times[t] + times[t′])/2)
    
    # Define the self-energies
    ΣL_u[t, t′] = U_t^2 .* GL_u[t, t′] .* GL_d[t, t′] .* transpose(GG_d[t′, t])
    ΣL_d[t, t′] = U_t^2 .* GL_u[t, t′] .* GL_d[t, t′] .* transpose(GG_u[t′, t])
    
    ΣG_u[t, t′] = U_t^2 .* GG_u[t, t′] .* GG_d[t, t′] .* transpose(GL_d[t′, t])
    ΣG_d[t, t′] = U_t^2 .* GG_u[t, t′] .* GG_d[t, t′] .* transpose(GL_u[t′, t])
end

second_Born! (generic function with 1 method)

In [12]:
tmax = 32;
atol = 1e-8
rtol = 1e-6;

In [13]:
@time sol = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> fd!(model, data, x...),
    [data.GL_u, data.GG_u, data.GL_d, data.GG_d],
    (0.0, tmax);
    callback = (x...) -> second_Born!(model, data, x...),
    atol = atol,
    rtol = rtol,
    stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

t: 0.0
t: 0.0
t: 0.0
t: 0.0
t: 8.548176969997932e-5
t: 0.00016801188879005504
t: 0.0002650495972796439
t: 0.00037531885268426795
t: 0.0009266651297073883
t: 0.0014228767790281966
t: 0.0039039350256322375
t: 0.016309226258652443
t: 0.02747398836837063
t: 0.058287492048922036
t: 0.11086371753919738
t: 0.1581823204804452
t: 0.20548765817758738
t: 0.25407422909357125
t: 0.3019560756927735
t: 0.3546132920822852
t: 0.40200478683284574
t: 0.4575086173713532
t: 0.5209451339098733
t: 0.5780379987945414
t: 0.6296033735722344
t: 0.6804374109066831
t: 0.7316491286977714
t: 0.7839044155331343
t: 0.8360277447211188
t: 0.8864088498511896
t: 0.9377681570084697
t: 0.9893465059975249
t: 1.0429914304357464
t: 1.0990408674051846
t: 1.1578986174694976
t: 1.2218103096725286
t: 1.2882327363218078
t: 1.3565958963551483
t: 1.4256448697701871
t: 1.4892153917908644
t: 1.4892153917908644
t: 1.5439566168910706
t: 1.59520116143549
t: 1.6444678641331887
t: 1.6974033074947488
t: 1.7526164247645895
t: 1.81293107219871

In [25]:
save("FH_3D_sol_U_"*string(U₀)*"_tmax_"*string(tmax)*"_atol_"*string(atol)*"_rtol_"*string(rtol)*".jld", "solution", sol)